# Dialogue Classifier with OpenAI and DSPy

In [2]:
import openai
import dspy

gpt4 = dspy.OpenAI(model = "gpt-4", max_tokens=4000)

dspy.settings.configure(lm=gpt4)

In [3]:
gpt4("say hello")

['Hello! How can I assist you today?']

In [14]:
def test_output(classification):
    valid_inputs = ["not relevant", "partial context", "full context"]
    return classification in valid_inputs

class DialogueClassifier(dspy.Signature):
    """Classify dialogue snippets into the given categories.
    
    The categories are given as:
    
    'not relevant': Completely unrelated to the question the user might eventually ask.
    'partial context': A question that gives only part of the required context, for example there are many different reports that could be relevant.
    'full context': The dialogue has everything required to answer the question
    """
    dialogue = dspy.InputField()
    classification = dspy.OutputField(desc="ONLY OUTPUT one of 'not relevant', 'partial context', or 'full context' NOTHING ELSE! IMPORTANT!")

class OutputParser(dspy.Signature):
    """Parse a raw response from a system into one of the desired outputs.
    
    The desired outputs are:
    
    -not relevant
    -partial context
    -full context
    """
    
    raw_response = dspy.InputField(desc="A raw response from a system that needs to be parsed.")
    desired_outputs = dspy.OutputField(desc="")

class ClassifierProgram(dspy.Module):
    def __init__(self):
        self.dialogue_classifier = dspy.ChainOfThought(DialogueClassifier)
        self.parser = dspy.Predict(OutputParser)
    
    def forward(self, dialogue):
        classification = self.dialogue_classifier(dialogue=dialogue).classification
        parser = self.parser(raw_response=classification).desired_outputs
        #[WIP] dspy.Suggest(test_output(parser), "One of 'not relevant', 'partial context', or 'full context'.")
        return dspy.Prediction(classification=parser)

In [15]:
my_program = ClassifierProgram()
my_program(dialogue="Hi Joel, how was your holiday").classification

'not relevant'

In [16]:
gpt4.inspect_history(n=1)





Parse a raw response from a system into one of the desired outputs.
    
    The desired outputs are:
    
    -not relevant
    -partial context
    -full context

---

Follow the following format.

Raw Response: A raw response from a system that needs to be parsed.
Desired Outputs:

---

Raw Response: not relevant
Desired Outputs: not relevant





In [18]:
my_program(dialogue="I have a problem with my Reports").classification

'partial context'

In [19]:
gpt4.inspect_history(n=1)





Parse a raw response from a system into one of the desired outputs.
    
    The desired outputs are:
    
    -not relevant
    -partial context
    -full context

---

Follow the following format.

Raw Response: A raw response from a system that needs to be parsed.
Desired Outputs:

---

Raw Response: partial context
Desired Outputs: partial context





In [20]:
my_program(dialogue="I have a issue creating the gardening report, can you help?").classification

'full context'

In [21]:
gpt4.inspect_history(n=1)





Parse a raw response from a system into one of the desired outputs.
    
    The desired outputs are:
    
    -not relevant
    -partial context
    -full context

---

Follow the following format.

Raw Response: A raw response from a system that needs to be parsed.
Desired Outputs:

---

Raw Response: full context
Desired Outputs: full context





In [ ]:
# optionally connect the WeaviateRM to your DSPy programs with:
'''
from dspy.retrieve.weaviate_rm import WeaviateRM
import weaviate
weaviate_client = weaviate.Client("http://localhost:8080")
retriever_model = WeaviateRM(collection_name="WeaviateBlogChunk", 
                            weaviate_collection_key="content",
                            weaviate_client=weaviate_client)
dspy.settings.configure(lm=llm, rm=retriever_model)
'''